In [33]:
# Backtested Hedge
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [34]:
# Define the ticker symbol
ticker_symbol = 'AAPL'

# Create a Ticker object
ticker = yf.Ticker(ticker_symbol)

# Get available options expiration dates
expiration_dates = ticker.options
print("Available expiration dates:", expiration_dates)

# Choose an expiration date
expiration_date = expiration_dates[0]  # Select the first available date

# Get the options chain for the chosen expiration date
options_chain = ticker.option_chain(expiration_date)

# Access call and put options data
calls = options_chain.calls
puts = options_chain.puts

# Display the first few rows of the calls and puts data
print("Call options data:")
print(calls.head())

print("\nPut options data:")
print(puts.head())


Available expiration dates: ('2024-12-06', '2024-12-13', '2024-12-20', '2024-12-27', '2025-01-03', '2025-01-10', '2025-01-17', '2025-02-21', '2025-03-21', '2025-04-17', '2025-06-20', '2025-07-18', '2025-08-15', '2025-09-19', '2025-12-19', '2026-01-16', '2026-06-18', '2026-12-18', '2027-01-15')
Call options data:
        contractSymbol             lastTradeDate  strike  lastPrice     bid  \
0  AAPL241206C00100000 2024-12-02 20:43:18+00:00   100.0     140.00  143.70   
1  AAPL241206C00125000 2024-12-03 20:31:38+00:00   125.0     117.19  118.50   
2  AAPL241206C00130000 2024-11-27 17:10:15+00:00   130.0     104.25  113.75   
3  AAPL241206C00135000 2024-11-27 16:06:35+00:00   135.0      99.55  108.80   
4  AAPL241206C00140000 2024-12-04 19:00:13+00:00   140.0     102.52  103.65   

      ask  change  percentChange  volume  openInterest  impliedVolatility  \
0  144.50     0.0            0.0       2             2           8.070317   
1  119.70     0.0            0.0       2             2   

In [35]:
def nearest_market_strike(strike, calls, puts, option_type=None):
    """
    Find the nearest available market strike price and associated information,
    along with counts of options for the specified type (call or put).
    
    Parameters:
    - strike: Desired strike price.
    - calls: DataFrame containing call options data.
    - puts: DataFrame containing put options data.
    - option_type: 'call', 'put', or None. If None, search both types.

    Returns:
    - A dictionary with the following keys:
        - 'strike': Nearest strike price.
        - 'sign': +1 for call, -1 for put.
        - 'type': 'call' or 'put'.
        - 'info': Additional option data (last price, bid, ask, implied volatility).
        - 'count': Total number of options available for the specified type.
    """
    # Filter options based on the specified type
    if option_type == 'call':
        relevant_options = calls.assign(option_type='call', sign=+1)
    elif option_type == 'put':
        relevant_options = puts.assign(option_type='put', sign=-1)
    else:  # If no specific type, combine both
        relevant_options = pd.concat([
            calls.assign(option_type='call', sign=+1),
            puts.assign(option_type='put', sign=-1)
        ], ignore_index=True)

    # Count the number of relevant options
    count = len(relevant_options)

    # Find the row with the closest strike
    relevant_options['strike_diff'] = abs(relevant_options['strike'] - strike)
    nearest_option = relevant_options.loc[relevant_options['strike_diff'].idxmin()]

    # Return relevant details
    return {
        'strike': nearest_option['strike'],
        'sign': nearest_option['sign'],
        'type': nearest_option['option_type'],
        'info': {
            'last_price': nearest_option['lastPrice'],
            'bid': nearest_option['bid'],
            'ask': nearest_option['ask'],
            'implied_volatility': nearest_option['impliedVolatility'],
            'open_interest': nearest_option['openInterest'],
        },
        'count': count
    }


ticker_symbol = 'AAPL'
expiration_date = '2024-12-06'
desired_strike = 107.5

# Create a Ticker object and retrieve the option chain
ticker = yf.Ticker(ticker_symbol)
options_chain = ticker.option_chain(expiration_date)
calls = options_chain.calls
puts = options_chain.puts

# Find the nearest call option
result_call = nearest_market_strike(desired_strike, calls, puts, option_type='call')
print(f"Nearest call option details for strike {desired_strike}:")
print(result_call)

# Find the nearest put option
result_put = nearest_market_strike(desired_strike, calls, puts, option_type='put')
print(f"\nNearest put option details for strike {desired_strike}:")
print(result_put)

# Find the nearest option (any type)
result_any = nearest_market_strike(desired_strike, calls, puts)
print(f"\nNearest option (any type) details for strike {desired_strike}:")
print(result_any)



Nearest call option details for strike 107.5:
{'strike': np.float64(100.0), 'sign': np.int64(1), 'type': 'call', 'info': {'last_price': np.float64(140.0), 'bid': np.float64(143.7), 'ask': np.float64(144.5), 'implied_volatility': np.float64(8.070317456054687), 'open_interest': np.int64(2)}, 'count': 50}

Nearest put option details for strike 107.5:
{'strike': np.float64(105.0), 'sign': np.int64(-1), 'type': 'put', 'info': {'last_price': np.float64(0.03), 'bid': np.float64(0.0), 'ask': np.float64(0.01), 'implied_volatility': np.float64(4.875003906249999), 'open_interest': np.int64(43)}, 'count': 47}

Nearest option (any type) details for strike 107.5:
{'strike': np.float64(105.0), 'sign': np.int64(-1), 'type': 'put', 'info': {'last_price': np.float64(0.03), 'bid': np.float64(0.0), 'ask': np.float64(0.01), 'implied_volatility': np.float64(4.875003906249999), 'open_interest': np.int64(43)}, 'count': 97}


In [36]:
# Modified expected value function:

def expected_value(wi, bi, Stm_1, r, sigma, dt, calls, puts):
    """
    Calculate E[max(wi * Stm + bi, 0) | St] based on signs of wi and bi.
    Constrains strike to nearest available market strikes and option data.
    """
    # Case 1: Forward contract scenario
    if wi >= 0 and bi >= 0:
        # Price of a forward contract = F = S_t * exp(r*dt)
        return wi * Stm_1 * np.exp(r * dt) + bi

    # Case 2: Call option scenario
    elif wi > 0 and bi < 0:
        # Infer strike from linear form: wi * S + bi => strike = -bi/wi
        strike = -bi / wi
        # Nearest available call strike
        result = nearest_market_strike(strike, calls, puts, option_type='call')
        chosen_strike = result['strike']
        implied_vol = result['info']['implied_volatility']
        # Ensure sigma is from market data now
        return wi * black_scholes(Stm_1, chosen_strike, r, implied_vol, dt, option_type='call')

    # Case 3: Put option scenario
    elif wi < 0 and bi > 0:
        # strike = -bi/wi
        strike = -bi / wi
        # Nearest available put strike
        result = nearest_market_strike(strike, calls, puts, option_type='put')
        chosen_strike = result['strike']
        implied_vol = result['info']['implied_volatility']
        # Note the original formula: we used -wi since wi < 0
        return -wi * black_scholes(Stm_1, chosen_strike, r, implied_vol, dt, option_type='put')

    # Case 4: Zero value
    elif wi <= 0 and bi <= 0:
        return 0.0